In [1]:
# Import Libraries
import os
import torch
from transformers import AutoTokenizer, AutoModel


LYRICS_FILES = '/Users/stamatiosorphanos/Documents/MultiModal-Deep-Learning/MultiModal/lyrics'

In [2]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

lyrics_data = []
lyrics_files = os.listdir(LYRICS_FILES)

for file_path in lyrics_files:
    if str(file_path).__contains__('.txt'):
        with open(os.path.join(LYRICS_FILES, file_path), "r") as f:
            lyrics = f.read()
            lyrics_data.append(lyrics)

encoded_data = tokenizer(lyrics_data, padding=True, truncation=True, max_length=512, return_tensors="pt")

with torch.no_grad():
    model_output = model(**encoded_data)
    embeddings = model_output.last_hidden_state[:, 0, :]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: The size of tensor a (1024) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
test_lyrics = "/Users/stamatiosorphanos/Documents/MultiModal-Deep-Learning/MultiModal/test.txt"

with open(test_lyrics, "r") as f:
    test_lyrics = f.read()

encoded_test = tokenizer(test_lyrics, padding=True, truncation=True, max_length=512, return_tensors="pt")

with torch.no_grad():
    test_output = model(**encoded_test)
    test_embedding = test_output.last_hidden_state[:, 0, :]

similarity_scores = torch.nn.functional.cosine_similarity(test_embedding, embeddings)
sorted_scores, sorted_indices = similarity_scores.sort(descending=True)

k = 5
top_k_scores = sorted_scores[:k]
top_k_indices = sorted_indices[:k]

print("Top {} similar songs:".format(k))
for score, index in zip(top_k_scores, top_k_indices):
    print("Song:", lyrics_files[index], "Score:", score.item())